# Deep Learning - Exercise 5/6 - Time Series Forecasting using RNN
This lecture is focused on the basic examples of the RNN usage for time series forecasting.

We will use Amazon stocks dataset from Yahoo finance. You can take look at this [this](https://finance.yahoo.com/quote/AMZN?p=AMZN)

Other datasets are also available, we will show you how to create your own as well.


[Open in Google colab](https://colab.research.google.com/github/jplatos/VSB-FEI-Deep-Learning/blob/master/dl_05_timeseries_rnn.ipynb)
[Download from Github](https://raw.githubusercontent.com/jplatos/VSB-FEI-Deep-Learning/main/dl_05_timeseries_rnn.ipynb)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.preprocessing import normalize, MinMaxScaler, StandardScaler # normalization of the matrix
import scipy
import pandas as pd

tf.version.VERSION

A utility function that show Learning progress.

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

# We have prepared three datasets for your experiments
## AAPL = Apple
## AMZN = Amazon
## SNE = Sony

## You are not limited to them, you can create your own datasets as well
## The prepared data covers period of the last 10 years with daily sampling frequency ~ 2500 values

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rasvob/2020-21-ARD/master/datasets/SNE.csv')
df.index = pd.to_datetime(df.Date)
df = df.drop('Date', axis=1)

In [ ]:
df.head()

# We are interested only in the Open column, which we will forecast
## We will deal with univariate time series forecasting in this lecture

In [ ]:
df = pd.DataFrame({'Price': df.iloc[:, 0]})

# The first step in every analysis task is taking a look at the data

In [ ]:
def show_timeseries(df):
    figsize = 20
    plt.figure(figsize=(figsize,figsize/2))
    plt.plot(df.index, df)
    plt.ylabel('Price ($)')
    plt.xlabel('Datetime')

# We can see that the stock price has significant trend and there are short term fluctuations as well

In [ ]:
show_timeseries(df)

# Let's take a look at data for the last month
## The changes in the short time periods are not as significant as in the lont term scenario

In [ ]:
viz_subset = df.iloc[-30:]
show_timeseries(viz_subset)

# We are usually interested in the dependency of the current value on the past values
## Auto-correlation function can help us with it
## We can vizualize the function values as well

In [ ]:
figsize = 20
plt.figure(figsize=(figsize,figsize/2))
plt.acorr(df.Price, maxlags=60)

# The time series has very high auto-correlation function values
## We can see that the ACF values are constantly lower for the longer lag values
## This means that we are mainly interested in the last few weeks/months of the data

# We worked with the classification models in the last few weeks
We evaluated quality of every created models based on its accuracy.

Accuracy is only one of the many metrics for the classification task but it is the simples one (take a look at the F1-Score, AuC/ROC, Precision/recall metrics if you are interested in this area).

Forecasting and regression tasks are not different - we have to evaluate model quality as well, butut now we use different types of metrics - most basic ones are MAE, RMSE which we already encountered.
There are many more metrics - R2, MAPE, sMAPE etc.

We have prepared the evaluation functions API for you.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

"""
Computes MAPE
"""
def mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

"""
Computes SMAPE
"""
def symetric_mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0))) * 100

"""
Computes MAE, MSE, MAPE, SMAPE, R2
"""
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    y_true, y_pred = df['y_true'].values, df['y_pred'].values
    return compute_metrics_raw(y_true, y_pred)

def compute_metrics_raw(y_true: pd.Series, y_pred: pd.Series) -> pd.DataFrame:
    mae, mse, mape, smape, r2 = mean_absolute_error(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), symetric_mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), r2_score(y_true=y_true, y_pred=y_pred)
    return pd.DataFrame.from_records([{'MAE': mae, 'MSE': mse, 'MAPE': mape, 'SMAPE': smape, 'R2': r2}], index=[0])

# The time series data are, surprisingly, time dependant
Time dependency means that we can't for example use Cross-validation or train/valid/test split without minding the time aspect. So to say - we can't split data randomly. We need to use test data from the period after the training set. If we don't take the time into account we will end up with leaking the data from test set to train one. Basically we let the model taking a look into the future. Which is obviously not wanted feature of the model, because there is no magic oracle available in the real-life forecasting scenarios.

# We will use the whole period of 2010 to 2018 as the training dataset
## First half of the 2019 will be used for validation and the second half as the testing dataset

In [ ]:
scaler = StandardScaler()
scaler.fit(df[df.index < '2019-01-01'].Price.values.reshape(-1,1))

In [ ]:
df['Price'] = scaler.transform(df.Price.values.reshape(-1,1))[:,0]

In [ ]:
df

# Our task will be to forecast the next day stock price

### Our first task is data preprocessing and feature creating - we will use past N values as the model input
#### We can tune this parameter - we will start with 2 months worth of the data

In [ ]:
maxlag = 60
df_orig = df.copy()

In [ ]:
for x in range(maxlag, 1, -1):
    df[f'Price_lag_{x}'] = df.Price.shift(x)

In [ ]:
df.head()

### We can see that now we have a lot of NaN values - we dont know values of the past prices before dataset start - we will have to drop this rows

# Now we will create the three datasets

In [ ]:
X, y = df[df.index < '2020-01-01'].iloc[:, 1:], df[df.index < '2020-01-01'].Price

In [ ]:
X_train, y_train = X[X.index.year < 2019], y[X.index.year < 2019]
X_valid, y_valid = X[(X.index.year == 2019) & (X.index.month <= 6)], y[(X.index.year == 2019) & (X.index.month <= 6)]
X_test, y_test = X[(X.index.year == 2019) & (X.index.month > 6)], y[(X.index.year == 2019) & (X.index.month > 6)]

In [ ]:
X_train = X_train.dropna()
y_train = y_train[X_train.index]

## The data has to have the same shapes in the first index

In [ ]:
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

### We can verify the defined intervals for the datasets

In [ ]:
X_train.head()

In [ ]:
print('TRAIN\n', y_train, '\nVALID\n',y_valid, '\nTEST\n',y_test)

In [ ]:
X_test_idx = X_test.index

# Now we can create the model and evaluate it

### We will start with the simple fully-connected network as baseline

In [ ]:
input_layer = keras.layers.Input(shape=X_train.shape[1])
x = keras.layers.Dense(64, activation='relu')(input_layer)
output_layer = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=['mae'])

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 32
epochs = 30
history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test.values)

In [ ]:
df_res = pd.DataFrame({'y_pred': y_pred.reshape(-1), 'y_true': y_test})

In [ ]:
df_res

In [ ]:
compute_metrics(df_res)

In [ ]:
def show_forecasts(df_res):
    figsize = 20
    plt.figure(figsize=(figsize,figsize/2))
    plt.plot(df_res.index, df_res.y_true, color='red')
    plt.plot(df_res.index, df_res.y_pred, color='green')
    plt.ylabel('Price ($)')
    plt.xlabel('Datetime')

# We can see that the error is quite high but even the relatively simple model captured overlaying trend of the data

In [ ]:
show_forecasts(df_res)

# Now we will try more complex model - recurrent one

The LSTM network expects the input data (X) to be provided with a specific array structure in the form of: [samples, time steps, features].

We can transform the prepared train and test input data into the expected structure using numpy.reshape()

In [ ]:
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)
X_valid = X_valid.values.reshape(X_valid.shape[0], X_valid.shape[1], 1)

In [ ]:
X_train.shape

In [ ]:
inp = keras.layers.Input(shape=X_train.shape[1:])
x = keras.layers.LSTM(64, activation='relu')(inp)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
output_layer = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inp, output_layer)
model.summary()
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=['mae'])

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 32
epochs = 30
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test)
df_res = pd.DataFrame({'y_pred': y_pred.reshape(-1), 'y_true': y_test}).dropna()

In [ ]:
compute_metrics(df_res)

In [ ]:
show_forecasts(df_res)

# We can stack multiple LSTM layers on each other

In [ ]:
inp = keras.layers.Input(shape=X_train.shape[1:])
x = keras.layers.LSTM(256, activation='relu', return_sequences=True)(inp)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LSTM(256, activation='relu', return_sequences=True)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Flatten(input_shape=(59, 256))(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inp, output_layer)
model.summary()
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=['mae'])

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 32
epochs = 50
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test)
# y_pred_diff = y_pred.reshape(-1) + price_orig[X_test_idx].shift(1)
df_res = pd.DataFrame({'y_pred': y_pred.reshape(-1), 'y_true': y_test}).dropna()

In [ ]:
compute_metrics(df_res)

In [ ]:
show_forecasts(df_res)

# And we can create really complex models based on LSTM - the situation with LSTM is the same as with every ANN in general - more complex model doesn't mean better solution

In [ ]:
inp = keras.layers.Input(shape=X_train.shape[1:])
x = keras.layers.LSTM(1024, activation='relu', return_sequences=True)(inp)
avg1 = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LSTM(256, activation='relu', return_sequences=True)(x)
avg2 = keras.layers.GlobalAveragePooling1D()(x)
concat = keras.layers.Concatenate()([keras.layers.Flatten()(avg1), keras.layers.Flatten()(avg2), keras.layers.Flatten()(x)])
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, activation='relu')(x)
output_layer = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inp, output_layer)
model.summary()
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=['mae'])

In [ ]:
# for the experimenting purpose only the EarlyStopping criterion is present
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 32
epochs = 50
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test)
df_res = pd.DataFrame({'y_pred': y_pred.reshape(-1), 'y_true': y_test}).dropna()

In [ ]:
compute_metrics(df_res)

In [ ]:
show_forecasts(df_res)

# We can see that the trend extrapolation can be an issue
## What if we could get rid of the trend?
The solution lies in the time series differencing - take a look at [this](https://otexts.com/fpp2/stationarity.html)

## We won't work with the stock price directly but we will use differences between two consecutive values
### Beware the forecast horizon length!
The differencing term can't be shorter than the horizon length, you would create dependency between two forecasts. We have forecast horizon of length 1 - only one day.

In [ ]:
maxlag = 60
df_diff = df_orig.copy()
df_diff.Price = df_diff.Price.diff(1)

In [ ]:
for x in range(1, maxlag):
    df_diff[f'Price_lag_{x}'] = df_diff.Price.shift(x)

In [ ]:
df_diff.head()

In [ ]:
show_timeseries(df_diff.Price)

In [ ]:
X, y = df_diff[df.index < '2020-01-01'].iloc[:, 1:], df_diff[df.index < '2020-01-01'].Price

In [ ]:
X_train, y_train = X[X.index.year < 2019], y[X.index.year < 2019]
X_valid, y_valid = X[(X.index.year == 2019) & (X.index.month <= 6)], y[(X.index.year == 2019) & (X.index.month <= 6)]
X_test, y_test = X[(X.index.year == 2019) & (X.index.month > 6)], y[(X.index.year == 2019) & (X.index.month > 6)]

In [ ]:
X_train = X_train.dropna()
y_train = y_train[X_train.index]

In [ ]:
print('TRAIN\n', y_train, '\nVALID\n',y_valid, '\nTEST\n',y_test)

In [ ]:
X_test_idx = X_test.index

In [ ]:
X_train = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])
X_valid = X_valid.values.reshape(X_valid.shape[0], 1, X_valid.shape[1])

In [ ]:
inp = keras.layers.Input(shape=X_train.shape[1:])
x = keras.layers.LSTM(1024, activation='relu', return_sequences=True)(inp)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.LSTM(512, activation='relu', return_sequences=True)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inp, output_layer)
model.summary()
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=['mae'])

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 32
epochs = 50
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test)
y_pred_diff = y_pred.reshape(-1) + df_orig.Price[X_test_idx].shift(1)
df_res = pd.DataFrame({'y_pred': y_pred_diff, 'y_true': df_orig.Price[X_test_idx]}).dropna()

# We can see that the LSTM alone is not a silver bullet - good preprocessing still matters even in the ANN area

In [ ]:
compute_metrics(df_res)

In [ ]:
show_forecasts(df_res)

# Task for the lecture

 - Choose other stock prices dataset
 - Try to create your own architecture using reccurent neural networks
 - Experiment a little - try different batch sizes, optimimizers, time lags as features, etc
 - Send me the Colab notebook with results and description what you did and your final solution!